In [2]:
%load_ext autoreload
%autoreload 2

import optuna
import pandas as pd
import lightgbm as lgb
import matplotlib.pyplot as plt
import shap
import numpy as np
import scipy
import seaborn as sns

from steps.prepare_data import load_split_processed_data, process_train_data, process_test_data
from utils.model import predict_booster, load_model, Metrics, predict_xgbm, predict

In [4]:
train_data, test_data = load_split_processed_data()

model_name = "LightGbmV2_new_total_features_fe_with_selection_v2_tuning_v2.pickle"

train_data_x = train_data.drop(columns=["target"])
train_data_y = train_data["target"]

test_data_x = test_data.drop(columns=["target"])
test_data_y = test_data["target"]

print("-----------Train----------")
predict_booster(model_name, train_data)

print("-----------Test----------")
predict_booster(model_name, test_data)

model_first, features_firsts = load_model(model_name)

In [16]:
train_data, test_data = load_split_processed_data()

model_name = "1_BEST_LightGbmV2_p071_r031_090auc_new_features.pickle"

train_data_x = train_data.drop(columns=["target"])
train_data_y = train_data["target"]

test_data_x = test_data.drop(columns=["target"])
test_data_y = test_data["target"]

print("-----------Train----------")
predict_booster(model_name, train_data)

print("-----------Test----------")
predict_booster(model_name, test_data)

model_second, features_second = load_model(model_name)

-----------Train----------

Metrics
AUC: 0.94
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.98    140413
           1       0.81      0.37      0.51      9585

    accuracy                           0.95    149998
   macro avg       0.88      0.68      0.74    149998
weighted avg       0.95      0.95      0.95    149998

-----------Test----------

Metrics
AUC: 0.90
Classification Report:
              precision    recall  f1-score   support

           0       0.96      0.99      0.97    140597
           1       0.71      0.31      0.43      9403

    accuracy                           0.95    150000
   macro avg       0.84      0.65      0.70    150000
weighted avg       0.94      0.95      0.94    150000



In [17]:
dataframe = test_data

features = features_firsts

X = dataframe[features]
y_true = dataframe.target

y_pred_proba_first = model_first.predict(X, num_iteration=model_first.best_iteration)
y_pred_proba_second = model_second.predict(X, num_iteration=model_second.best_iteration)

y_pred_proba = 0.4 * y_pred_proba_first + 0.6 * y_pred_proba_second

threshold = 0.5
y_pred = (y_pred_proba >= threshold).astype(int)

Metrics().call(y_true, y_pred, y_pred_proba)


Metrics
AUC: 0.90
Classification Report:
              precision    recall  f1-score   support

           0       0.97      0.97      0.97    140597
           1       0.52      0.53      0.52      9403

    accuracy                           0.94    150000
   macro avg       0.75      0.75      0.75    150000
weighted avg       0.94      0.94      0.94    150000

